<a href="https://colab.research.google.com/github/raki-rankawat/stm32/blob/main/VWW_STM32_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install onnx onnxruntime onnxscript onnxruntime-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00


In [2]:
import os
import time
import tarfile
import random
import shutil
from pathlib import Path
from urllib.request import urlretrieve

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import onnx
import onnxruntime as ort

from onnxruntime.quantization import quantize_static, QuantType, QuantFormat, CalibrationDataReader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# ----------------------------
# Auto Download + Prepare VWW (10k subset)
# ----------------------------
vww_url = "https://www.silabs.com/public/files/github/machine_learning/benchmarks/datasets/vw_coco2014_96.tar.gz"

base_dir = Path("/content/vww_work")
archive_path = base_dir / "vw_coco2014_96.tar.gz"
extract_dir = base_dir / "extracted"
subset_dir = base_dir / "vww_10k"

# Subset config: 5k person + 5k non_person
n_per_class = 5000
val_ratio = 0.20

random.seed(41)
torch.manual_seed(41)

def download_vww():
    base_dir.mkdir(parents=True, exist_ok=True)

    if archive_path.exists() and archive_path.stat().st_size > 0:
        print("✅ VWW archive already downloaded")
        return

    print("⬇️ Downloading VWW archive...")
    urlretrieve(vww_url, archive_path)
    print("✅ Download complete:", archive_path)

def extract_vww():
    extract_dir.mkdir(parents=True, exist_ok=True)

    if any(extract_dir.iterdir()):
        print("✅ VWW already extracted")
        return

    print("📦 Extracting VWW archive...")
    with tarfile.open(archive_path, "r:gz") as tar:
        tar.extractall(extract_dir)
    print("✅ Extraction complete:", extract_dir)

def find_vww_root():
    # Find folder that contains BOTH person/ and non_person/
    for p in extract_dir.rglob("person"):
        if p.is_dir() and (p.parent / "non_person").is_dir():
            return p.parent
    raise RuntimeError("❌ Could not find 'person' and 'non_person' directories under extracted dataset")

def list_images(folder):
    exts = {".jpg", ".jpeg", ".png"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def make_vww_subset(src_root):
    # Skip if subset already exists
    if (subset_dir / "train" / "person").is_dir() and (subset_dir / "val" / "non_person").is_dir():
        print("✅ VWW 10k subset already exists:", subset_dir)
        return

    for split in ["train", "val"]:
        for c in ["person", "non_person"]:
            (subset_dir / split / c).mkdir(parents=True, exist_ok=True)

    person_imgs = list_images(src_root / "person")
    nonperson_imgs = list_images(src_root / "non_person")

    if len(person_imgs) < n_per_class or len(nonperson_imgs) < n_per_class:
        raise ValueError(
            f"❌ Not enough images:\n"
            f"person: {len(person_imgs)} (need {n_per_class})\n"
            f"non_person: {len(nonperson_imgs)} (need {n_per_class})"
        )

    random.shuffle(person_imgs)
    random.shuffle(nonperson_imgs)

    person_sel = person_imgs[:n_per_class]
    nonperson_sel = nonperson_imgs[:n_per_class]

    def split_list(lst, val_ratio):
        n_val = int(len(lst) * val_ratio)
        return lst[n_val:], lst[:n_val]  # train, val

    p_train, p_val = split_list(person_sel, val_ratio)
    n_train, n_val = split_list(nonperson_sel, val_ratio)

    def copy_files(files, dst_dir):
        for f in files:
            dst = dst_dir / f.name
            # avoid rare collisions
            if dst.exists():
                dst = dst_dir / (f"{f.parent.name}_{f.name}")
            shutil.copy2(f, dst)

    print("🧩 Creating VWW 10k subset...")
    copy_files(p_train, subset_dir / "train" / "person")
    copy_files(p_val,   subset_dir / "val"   / "person")
    copy_files(n_train, subset_dir / "train" / "non_person")
    copy_files(n_val,   subset_dir / "val"   / "non_person")
    print("✅ VWW subset created at:", subset_dir)

download_vww()
extract_vww()
vww_root = find_vww_root()
print("✅ Found VWW root:", vww_root)
make_vww_subset(vww_root)

⬇️ Downloading VWW archive...
✅ Download complete: /content/vww_work/vw_coco2014_96.tar.gz
📦 Extracting VWW archive...


/tmp/ipython-input-4249398104.py:38: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(extract_dir)


✅ Extraction complete: /content/vww_work/extracted
✅ Found VWW root: /content/vww_work/extracted/vw_coco2014_96
🧩 Creating VWW 10k subset...
✅ VWW subset created at: /content/vww_work/vww_10k


In [6]:
# ----------------------------
# VWW Dataset (same idea as before: use your local vww_10k subset)
# ----------------------------
base_dir = Path("/content/vww_work")
subset_dir = base_dir / "vww_10k"

batch_size = 1
img_size = 96

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

# IMPORTANT: use your VAL split folder
test_dataset = datasets.ImageFolder(root=str(subset_dir / "val"), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print("Class mapping:", test_dataset.class_to_idx)

Class mapping: {'non_person': 0, 'person': 1}


In [7]:
# ----------------------------
# VWW Model Definition (must match training)
# ----------------------------
class VWWConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 2)

        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) # 96 -> 48

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # 48 -> 24

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2) # 24 -> 12

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # 12 -> 6

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        if self.training:
            x = self.dropout(x)
        x = self.fc2(x)

        return x

In [8]:
# ----------------------------
# Load weights (BEST VWW checkpoint)
# ----------------------------
model = VWWConvNet()
model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/stm_vww_best.pth", map_location=torch.device("cpu")))
print("✅ Loaded stm_vww_best.pth")

✅ Loaded stm_vww_best.pth


In [9]:
# ----------------------------
# PyTorch Accuracy on first N samples (same style)
# ----------------------------
model.eval()
correct = 0
total = 0

N = 200

with torch.no_grad():
    for i, (x, y) in enumerate(test_loader):
        if i >= N:
            break
        out = model(x)
        pred = out.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

print("PyTorch accuracy on first 200:", 100 * correct / total)

PyTorch accuracy on first 200: 83.5


### FP32 Pipeline

In [11]:
# Export To ONNX
def export_onnx(model, onnx_path):
    model.eval()
    dummy = torch.randn(1, 3, 96, 96)  # NCHW

    torch.onnx.export(
        model,
        dummy,
        onnx_path,
        input_names=["input"],
        output_names=["logits"],
        export_params=True,
        opset_version=18,
        do_constant_folding=True,
        dynamic_axes={"input": {0: "batch_size"}, "logits": {0: "batch_size"}},
        dynamo=False
    )
    onnx.checker.check_model(onnx_path, full_check=False)
    print(f"ONNX model saved to: {onnx_path}")

export_onnx(model, "vww_convnet_fp32.onnx")

ONNX model saved to: vww_convnet_fp32.onnx


/tmp/ipython-input-516851722.py:6: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [12]:
# Collect N samples (SAVE NCHW for ST to avoid any internal NHWC->NCHW conversion)
model.eval()

inputs_nchw = []
logits = []
labels = []

with torch.no_grad():
    for i, (x, y) in enumerate(test_loader):
        if i >= N:
            break

        out = model(x)

        # Keep NCHW (1,3,96,96) -> store (3,96,96)
        x_nchw = x.detach().cpu().numpy().astype(np.float32)         # (1,3,96,96)
        out_np = out.detach().cpu().numpy()[0].astype(np.float32)    # (2,)

        inputs_nchw.append(x_nchw[0])    # (3,96,96)
        logits.append(out_np)            # (2,)
        labels.append(int(y.item()))

inputs_nchw = np.stack(inputs_nchw, axis=0)     # (N,3,96,96)
logits = np.stack(logits, axis=0)               # (N,2)
labels = np.array(labels, dtype=np.int32)       # (N,)

np.savez("vww_val_200_io.npz", input=inputs_nchw, logits=logits)
np.savez("vww_labels_200.npz", label=labels)

print("Saved input shape:", inputs_nchw.shape, "min/max:", inputs_nchw.min(), inputs_nchw.max())

Saved input shape: (200, 3, 96, 96) min/max: -2.117904 2.64


In [13]:
# Compute Accuracy (STM output NPZ)
def compute_accuracy(
    labels_npz_path,
    outputs_npz_path,
    output_key="c_outputs_1",
    num_classes=2,
    as_percentage=False
):
    labels = np.load(labels_npz_path)["label"].astype(np.int64)
    out = np.load(outputs_npz_path)

    logits = out[output_key].reshape(len(labels), num_classes)
    pred = np.argmax(logits, axis=1)

    acc = (pred == labels).mean()
    return acc * 100 if as_percentage else acc

In [15]:
# Example (after you run STM and get network_val_io.npz)
acc = compute_accuracy("vww_labels_200.npz", "network_val_io.npz", output_key="c_outputs_1", num_classes=2, as_percentage=True)
print("STM32 accuracy:", acc)

STM32 accuracy: 83.5


### INT8 Pipeline

In [16]:
# Calibration NPZ (inputs only)
def make_calib_npz(test_dataset, N=200, out_path="vww_calib_200.npz"):
    loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    xs = []
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            if i >= N:
                break
            xs.append(x.detach().cpu().numpy()[0].astype(np.float32))  # (3,96,96)

    xs = np.stack(xs, axis=0)  # (N,3,96,96)
    np.savez(out_path, input=xs)
    print("Saved calib:", out_path, xs.shape)
    return out_path

In [17]:
class CalibReader(CalibrationDataReader):
    def __init__(self, npz_path, input_name="input"):
        self.x = np.load(npz_path)["input"].astype(np.float32)
        self.input_name = input_name
        self.i = 0

    def get_next(self):
        if self.i >= len(self.x):
            return None
        batch = self.x[self.i:self.i+1]  # (1,3,96,96)
        self.i += 1
        return {self.input_name: batch}

In [18]:
def quantize_int8_qdq(fp32_onnx="vww_convnet_fp32.onnx",
                      calib_npz="vww_calib_200.npz",
                      int8_onnx="vww_convnet_int8_static_qdq.onnx"):
    reader = CalibReader(calib_npz, input_name="input")

    quantize_static(
        model_input=fp32_onnx,
        model_output=int8_onnx,
        calibration_data_reader=reader,
        quant_format=QuantFormat.QDQ,
        activation_type=QuantType.QInt8,
        weight_type=QuantType.QInt8,
        per_channel=True,
    )
    print("Saved INT8:", int8_onnx)
    return int8_onnx

In [19]:
calib_npz = make_calib_npz(test_dataset, N=200, out_path="vww_calib_200.npz")
quantize_int8_qdq("vww_convnet_fp32.onnx", calib_npz, "vww_convnet_int8_static_qdq.onnx")

Saved calib: vww_calib_200.npz (200, 3, 96, 96)


Saved INT8: vww_convnet_int8_static_qdq.onnx


'vww_convnet_int8_static_qdq.onnx'

In [20]:
acc = compute_accuracy("vww_labels_200.npz", "network_val_io.npz", output_key="c_outputs_1", num_classes=2, as_percentage=True)
print("STM32 accuracy:", acc)

STM32 accuracy: 84.5
